In [67]:
# %% Imports
import pandas as pd
import missingno as msno
import numpy as np

# Load in Raw Data
yelp = pd.read_csv(f"../data/yelp_entries_gacities.csv")
print(yelp.shape)
yelp.drop_duplicates(subset='alias', keep="first", inplace=True)
print(yelp.shape)
yelp.reset_index()
yelp = yelp.drop(columns=['zip_code'])

(4026, 17)
(1778, 17)


In [68]:
# This goes to all of the dictionaries stored as objects, evaluates them, and extracts their values as columns
coordinates_list = yelp.coordinates.values.tolist()
coordinates_df = pd.DataFrame([eval(x) for x in coordinates_list])

location_list = yelp.location.values.tolist()
location_df = pd.DataFrame([eval(x) for x in location_list])
categories_list = yelp.categories.values.tolist()
category_df = pd.DataFrame([eval(x) for x in categories_list], columns=["category_1",
                                                                "category_2",
                                                                "category_3",
                                                                ])

location_df.drop(labels=['address3', 'address2'], axis=1, inplace = True) # Droping the extra address columns

# Categories needed some extra love (they were stored in nested dictionaries)
category_df["category_1"] = category_df['category_1'].map(lambda x: x['alias'])
category_df["category_2"] = category_df['category_2'].map(lambda x: x['alias'] if type(x) == dict else x)
category_df["category_3"] = category_df['category_3'].map(lambda x: x['alias'] if type(x) == dict else x)

# Combining all the datasets
df = yelp.join(coordinates_df)
df = df.join(location_df)
df = df.join(category_df)

# Droping anything that will not be helpful
df = df[df['state'] == 'GA']
df= df[['name', 
        'price', 
        'rating', 'review_count',
        'latitude','longitude', 
        'city', 'state', 'zip_code',
        'category_1', 'category_2', 'category_3']]
# To CSV
df.to_csv('../data/yelp_gacities_clean.csv', index=False)

## Code for combining X and y:

In [73]:
## Read in Data and drop any rows missing location values ###
yelp_clean_df = pd.read_csv("../data/yelp_gacities_clean.csv") # This line is unessessary now.
income_df = pd.read_csv('../data/zips_list.csv')
yelp_clean_df.dropna(subset=['zip_code', 'latitude', 'longitude'], inplace=True)

In [74]:
# ### I've also added a variable to deteremine if it is a park or not, just for fun
# ## it does not work anymore
# yelp_clean_df['is_park'] = np.where(np.logical_and(yelp_clean_df['name'].str.lower().str.contains("park"), 
#                                              yelp_clean_df['price'].isna()), 1, 0)
# yelp_clean_df[yelp_clean_df['is_park'] == 1]

In [75]:
## This is the actual combining part:
cdf = pd.merge(yelp_clean_df, income_df, on="zip_code")
# I'm also ordinalizing the price data feel free to suggest something else, 
# most of the data without pricing was free or an activity so I believe this works(open to suggestions)
recode = {np.nan : 0, '$' : 1, '$$' : 2, '$$$' : 3, '$$$$' : 4 }
cdf['price'].replace(recode, inplace=True) # Recode
# To CSV
cdf.to_csv('../data/yelp_income_gacities_clean.csv', index= False)